In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from lib import Datasets
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [4]:
# model_name = "meta-llama/Llama-2-7b-hf"
model_name = "distilgpt2"
# model_name = "roberta-base"
device = 'cuda:0'

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)
# model_name = "roberta-base"

# model_checkpoint = "NousResearch/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)

for name, p in model.named_parameters():
    print(name, p.shape)

2025-02-11 19:31:44.785814: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-11 19:31:44.807891: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739291504.836946 3414517 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739291504.845767 3414517 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-11 19:31:44.876628: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [5]:
b = 8
# lm_datasets = Datasets.LoadWikiOther(tokenizer, b)

In [6]:
# print(lm_datasets)

In [7]:
# import torch
# from lib import Train

# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Train.train_model(model, lm_datasets, optimizer, tokenizer)


In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    f"./workout/keke-test",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=10,
    logging_dir=f"./logs/kekek-test",
    push_to_hub=False,
    save_strategy="epoch",
    # use_cpu=True,
    # bf16=True,
)

In [9]:
lm_datasets = Datasets.LoadWiki(tokenizer, 10, 128)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

Map (num_proc=8):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [10]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries , along with Valkyria Chronicles II director Takeshi Oz'

In [11]:
# trainer.train()

In [12]:


# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [13]:
import math
import time

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.4f}")

# while True:
#     try:
#         eval_results = trainer.evaluate()
#         print(f"Perplexity: {math.exp(eval_results['eval_loss']):.4f}")
#         break
#     except Exception as e:
#         print("Вы обосрались")
#         print(e)
#         time.sleep(1)

Perplexity: 86.2311
